# Olym+ BlazePose Pipeline (Colab)
End-to-end: install → mount Drive → datasets → landmarks → features → windows → train → reps → figures.

In [ ]:
# 0) Setup
!pip install mediapipe opencv-python-headless pandas numpy scikit-learn xgboost matplotlib seaborn tqdm pyarrow joblib -q
from google.colab import drive
drive.mount('/content/drive')
import os
BASE='/content/drive/MyDrive/olymp_project'
for d in ['data/mmfit_videos','data/kaggle_pushup','landmarks','features','models','results']:
    os.makedirs(f'{BASE}/{d}', exist_ok=True)
print('Workdir:', BASE)

## 1) Datasets
- Upload videos to Drive folders:
  - mmfit videos → /content/drive/MyDrive/olymp_project/data/mmfit_videos/
  - kaggle pushup → /content/drive/MyDrive/olymp_project/data/kaggle_pushup/
- Optional: use Kaggle API (place kaggle.json in Drive root and run below).

In [ ]:
# Kaggle optional
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/ 2>/dev/null || true
!chmod 600 ~/.kaggle/kaggle.json 2>/dev/null || true
# Example: replace <dataset-slug> with the dataset you want
# !kaggle datasets download -d <dataset-slug> -p $BASE/data/kaggle_pushup --unzip

In [ ]:
# 2) Landmark extraction (MediaPipe BlazePose)
import cv2, mediapipe as mp, os, csv, glob, tqdm
mp_pose = mp.solutions.pose
def extract_landmarks_from_video(video_path, out_csv, sample_rate=2):
    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(video_path)
    frame_i = 0
    os.makedirs(os.path.dirname(out_csv), exist_ok=True)
    with open(out_csv, 'a', newline='') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            header = ['video','frame','timestamp','width','height']
            for i in range(33): header += [f'lm{i}_x',f'lm{i}_y',f'lm{i}_z',f'lm{i}_vis']
            writer.writerow(header)
        while cap.isOpened():
            ret,frame = cap.read()
            if not ret: break
            if frame_i % sample_rate == 0:
                h,w = frame.shape[:2]
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                res = pose.process(img)
                row = [os.path.basename(video_path), frame_i, cap.get(cv2.CAP_PROP_POS_MSEC), w, h]
                if res.pose_landmarks:
                    for lm in res.pose_landmarks.landmark:
                        row += [lm.x, lm.y, lm.z, lm.visibility]
                else:
                    row += [None]*(33*4)
                writer.writerow(row)
            frame_i += 1
    cap.release()
base=f'{BASE}/data'
out_dir=f'{BASE}/landmarks'
for dataset_folder in ['mmfit_videos','kaggle_pushup']:
    infolder=os.path.join(base,dataset_folder)
    outcsv=os.path.join(out_dir,f'{dataset_folder}_landmarks.csv')
    vids=glob.glob(os.path.join(infolder,'**','*.mp4'), recursive=True)
    print('Processing', len(vids), 'videos in', infolder)
    for v in tqdm.tqdm(vids):
        extract_landmarks_from_video(v, outcsv, sample_rate=2)
print('Done landmarks')

In [ ]:
# 3) Feature engineering (per-frame)
import pandas as pd, numpy as np, math
LANDMARK_CSV=f'{BASE}/landmarks/mmfit_videos_landmarks.csv'
OUT_PKL=f'{BASE}/features/mmfit_features.pkl'
df=pd.read_csv(LANDMARK_CSV)
df=df.dropna(subset=['lm0_x']).reset_index(drop=True)
def get_landmarks(row):
    lm=[]
    for i in range(33):
        lm.append((row.get(f'lm{i}_x',np.nan), row.get(f'lm{i}_y',np.nan), row.get(f'lm{i}_z',np.nan)))
    return np.array(lm,float)
def angle_between(a,b,c):
    ba=a-b; bc=c-b
    if np.any(np.isnan(ba)) or np.any(np.isnan(bc)): return np.nan
    cosang = float(np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc)+1e-9))
    cosang = max(-1., min(1., cosang))
    return math.degrees(math.acos(cosang))
idx={'nose':0,'l_sh':11,'r_sh':12,'l_el':13,'r_el':14,'l_wr':15,'r_wr':16,'l_hip':23,'r_hip':24,'l_knee':25,'r_knee':26,'l_ank':27,'r_ank':28}
rows=[]
for _,r in df.iterrows():
    lm=get_landmarks(r); w,h=r['width'], r['height']; pts=lm.copy(); pts[:,0]*=w; pts[:,1]*=h
    l_elbow=angle_between(pts[idx['l_sh']], pts[idx['l_el']], pts[idx['l_wr']])
    r_elbow=angle_between(pts[idx['r_sh']], pts[idx['r_el']], pts[idx['r_wr']])
    l_knee=angle_between(pts[idx['l_hip']], pts[idx['l_knee']], pts[idx['l_ank']])
    r_knee=angle_between(pts[idx['r_hip']], pts[idx['r_knee']], pts[idx['r_ank']])
    l_hip=angle_between(pts[idx['l_sh']], pts[idx['l_hip']], pts[idx['l_knee']])
    r_hip=angle_between(pts[idx['r_sh']], pts[idx['r_hip']], pts[idx['r_knee']])
    mid_hip=(pts[idx['l_hip']]+pts[idx['r_hip']])/2.0; nose=pts[idx['nose']]
    body_tilt=(np.degrees(np.arctan2((nose-mid_hip)[1], (nose-mid_hip)[0])) if not np.any(np.isnan(nose-mid_hip)) else np.nan)
    def dist(a,b):
        if np.any(np.isnan(a[:2])) or np.any(np.isnan(b[:2])): return np.nan
        return float(np.linalg.norm(a[:2]-b[:2]))
    shoulder_width=dist(pts[idx['l_sh']], pts[idx['r_sh']]); hip_width=dist(pts[idx['l_hip']], pts[idx['r_hip']])
    torso_len=dist((pts[idx['l_sh']]+pts[idx['r_sh']])/2.0, mid_hip); arm_len=dist(pts[idx['l_sh']], pts[idx['l_wr']])
    rows.append({'video':r['video'],'frame':int(r['frame']),'timestamp':float(r['timestamp']),
                 'l_elbow':l_elbow,'r_elbow':r_elbow,'l_knee':l_knee,'r_knee':r_knee,'l_hip':l_hip,'r_hip':r_hip,
                 'body_tilt':body_tilt,'ratio_sh_hip': shoulder_width/(hip_width+1e-6) if not np.isnan(shoulder_width) and not np.isnan(hip_width) else np.nan,
                 'ratio_arm_torso': arm_len/(torso_len+1e-6) if not np.isnan(arm_len) and not np.isnan(torso_len) else np.nan,
                 'shoulder_width': shoulder_width, 'hip_width': hip_width})
feat_df=pd.DataFrame(rows)
feat_df.to_pickle(OUT_PKL); print('Saved', OUT_PKL, feat_df.shape)

In [ ]:
# 4) Windowing + labels
import pandas as pd, numpy as np, os
feat_df=pd.read_pickle(f'{BASE}/features/mmfit_features.pkl')
labels_csv=f'{BASE}/data/video_labels.csv'
if not os.path.exists(labels_csv):
    with open(labels_csv,'w') as f: f.write('video,exercise,form
example1.mp4,biceps,correct
')
labels=pd.read_csv(labels_csv)
feat_df=feat_df.merge(labels, on='video', how='left')
FPS=30; WINDOW_SEC=1.5; WINDOW_SIZE=int(WINDOW_SEC*FPS/2); STEP=WINDOW_SIZE//2
rows=[]
num_cols=['l_elbow','r_elbow','l_knee','r_knee','l_hip','r_hip','body_tilt','ratio_sh_hip','ratio_arm_torso']
for v, vdf in feat_df.groupby('video'):
  vdf=vdf.sort_values('frame').reset_index(drop=True)
  for s in range(0, len(vdf)-WINDOW_SIZE+1, STEP):
    w=vdf.iloc[s:s+WINDOW_SIZE]; agg={'video':v,'start_idx':int(w.iloc[0]['frame']),'exercise':w.get('exercise',pd.Series([None])).iloc[0],'form':w.get('form',pd.Series([None])).iloc[0]}
    for c in num_cols:
      agg[f'{c}_mean']=float(w[c].mean()); agg[f'{c}_std']=float(w[c].std()); agg[f'{c}_min']=float(w[c].min()); agg[f'{c}_max']=float(w[c].max())
    rows.append(agg)
win_df=pd.DataFrame(rows)
win_df.to_pickle(f'{BASE}/features/mmfit_windows.pkl'); win_df.head()

In [ ]:
# 5) Train RandomForest baseline vs enhanced
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import joblib, os
df=pd.read_pickle(f'{BASE}/features/mmfit_windows.pkl').dropna(subset=['exercise']).reset_index(drop=True)
le=LabelEncoder(); df['y']=le.fit_transform(df['exercise'])
videos=df['video'].unique(); tr,te=train_test_split(videos, test_size=0.2, random_state=42)
trdf=df[df.video.isin(tr)]; tedf=df[df.video.isin(te)]
baseline=['l_elbow_mean','r_elbow_mean','l_knee_mean','r_knee_mean','l_hip_mean','r_hip_mean']
enh=baseline+['body_tilt_mean','ratio_sh_hip_mean','ratio_arm_torso_mean']+[c for c in df.columns if c.endswith('_std')]
rfb=RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1).fit(trdf[baseline].fillna(0), trdf['y'])
rfe=RandomForestClassifier(n_estimators=300, max_depth=20, random_state=42, n_jobs=-1).fit(trdf[enh].fillna(0), trdf['y'])
pb=rfb.predict(tedf[baseline].fillna(0)); pe=rfe.predict(tedf[enh].fillna(0))
print('Baseline:
', classification_report(tedf['y'], pb, target_names=le.classes_))
print('Enhanced:
', classification_report(tedf['y'], pe, target_names=le.classes_))
cm_b=confusion_matrix(tedf['y'], pb); cm_e=confusion_matrix(tedf['y'], pe)
plt.figure(figsize=(10,4)); plt.subplot(1,2,1); sns.heatmap(cm_b, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_); plt.title('Baseline')
plt.subplot(1,2,2); sns.heatmap(cm_e, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_); plt.title('Enhanced'); plt.tight_layout()
plt.savefig(f'{BASE}/results/confusion_comparison.png', dpi=200)
joblib.dump(rfb, f'{BASE}/models/rf_baseline.pkl'); joblib.dump(rfe, f'{BASE}/models/rf_enhanced.pkl')
print('Saved models and figures to Drive/results & Drive/models')

In [ ]:
# 6) Rep counting example (biceps using left elbow)
import pandas as pd
ff=pd.read_pickle(f'{BASE}/features/mmfit_features.pkl')
v=ff['video'].unique()[0] if len(ff)>0 else None
def count(ang, down=50, up=160):
  s='up'; c=0
  for a in ang:
    if s=='up' and a<down: s='down'
    elif s=='down' and a>up: s='up'; c+=1
  return c
if v:
  vdf=ff[ff.video==v].sort_values('frame'); print('Video:', v, 'Reps:', count(vdf['l_elbow'].fillna(method='ffill').fillna(method='bfill').values))
else:
  print('No features yet')

# BlazePose Exercise Classifier — Google Colab Pipeline

Follow the cells top-to-bottom. Use GPU runtime if you plan to process many videos, though CPU is fine for this pipeline.

## 1. Select Runtime and Verify Environment

- In Colab, go to Runtime → Change runtime type → Hardware accelerator: GPU (optional).
- Run the cell below to print Python and platform.

In [ ]:
# Environment check
import sys, platform
print(sys.version)
print(platform.platform())

## 2. Install Dependencies
This installs the core libraries used in the pipeline.

In [ ]:
!pip install -q mediapipe opencv-python-headless numpy pandas scikit-learn matplotlib seaborn tqdm joblib

## 3. Mount Google Drive
This persists datasets and outputs so they survive Colab sessions.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
BASE_DIR = '/content/drive/MyDrive/blazepose-classifier'
SRC_DIR = f'{BASE_DIR}/src'
DATA_DIR = f'{BASE_DIR}/data'
LANDMARKS_DIR = f'{BASE_DIR}/landmarks'
FEATURES_DIR = f'{BASE_DIR}/features'
MODELS_DIR = f'{BASE_DIR}/models'
RESULTS_DIR = f'{BASE_DIR}/results'

for d in [BASE_DIR, SRC_DIR, DATA_DIR, LANDMARKS_DIR, FEATURES_DIR, MODELS_DIR, RESULTS_DIR]:
    os.makedirs(d, exist_ok=True)

print('BASE_DIR =', BASE_DIR)

## 4. Acquire Project Code (clone/upload)
Place your project files in Drive under BASE_DIR. If this is your first run, upload the `src/` folder to `blazepose-classifier/src` or clone your repo.

In [ ]:
# Verify that required scripts are present; if not, create quick copies from this notebook (optional)
import os
required_files = ['extract_landmarks.py','feature_engineering.py','windowing.py','train_models.py','rep_counting.py']
missing = [f for f in required_files if not os.path.exists(f'{SRC_DIR}/{f}')]
print('Missing in SRC_DIR:', missing)
print('SRC_DIR =', SRC_DIR)
print('DATA_DIR =', DATA_DIR)


## 5. (Option A) Extract Landmarks from raw videos
If you uploaded `.mp4` videos into Drive under `data/mmfit_videos` or `data/kaggle_pushup`, run this. Skip if you only have MM-Fit `pose_2d.npy`.

In [ ]:
# Option A: Extract from videos (requires SRC_DIR/extract_landmarks.py present)
!python {SRC_DIR}/extract_landmarks.py --input {DATA_DIR}/mmfit_videos {DATA_DIR}/kaggle_pushup --output {LANDMARKS_DIR} --name mmfit_videos_landmarks.csv --sample-rate 2

## 6. (Option B) Convert MM-Fit pose_2d.npy → landmarks CSV
If you have MM-Fit `wXX/wXX_pose_2d.npy`, convert them into the same landmarks CSV expected by the pipeline.

In [ ]:
# Option B: Convert pose_2d.npy to landmarks CSV using an inline converter
import os, csv, glob
import numpy as np

BLAZEPOSE_SIZE = 33
COCO17 = {'nose':0,'left_eye':1,'right_eye':2,'left_ear':3,'right_ear':4,'left_shoulder':5,'right_shoulder':6,'left_elbow':7,'right_elbow':8,'left_wrist':9,'right_wrist':10,'left_hip':11,'right_hip':12,'left_knee':13,'right_knee':14,'left_ankle':15,'right_ankle':16}
OPENPOSE25 = {'nose':0,'neck':1,'right_shoulder':2,'right_elbow':3,'right_wrist':4,'left_shoulder':5,'left_elbow':6,'left_wrist':7,'mid_hip':8,'right_hip':9,'right_knee':10,'right_ankle':11,'left_hip':12,'left_knee':13,'left_ankle':14,'right_eye':15,'left_eye':16,'right_ear':17,'left_ear':18}
TARGET = {'nose':0,'left_shoulder':11,'right_shoulder':12,'left_elbow':13,'right_elbow':14,'left_wrist':15,'right_wrist':16,'left_hip':23,'right_hip':24,'left_knee':25,'right_knee':26,'left_ankle':27,'right_ankle':28}
SCHEMAS = {'coco17':COCO17,'openpose25':OPENPOSE25}

def infer_schema_from_npy(path):
    arr = np.load(path)
    if arr.ndim == 3:
        K = arr.shape[1]
    elif arr.ndim == 2:
        K = arr.shape[1]//2
    else:
        return None
    if K == 17:
        return 'coco17'
    if K == 25:
        return 'openpose25'
    return None

def build_mapping(schema):
    src = SCHEMAS[schema]
    mapping = {}
    for name in TARGET:
        if name in src:
            mapping[src[name]] = TARGET[name]
    return mapping

def write_header(w):
    header=['video','frame','timestamp','width','height']
    for i in range(BLAZEPOSE_SIZE):
        header += [f'lm{i}_x',f'lm{i}_y',f'lm{i}_z',f'lm{i}_vis']
    w.writerow(header)

# find pose_2d files
pose_files = sorted(glob.glob(f"{DATA_DIR}/mmfit_videos/mm-fit/mm-fit/w*/w*_pose_2d.npy"))
print('Found pose_2d files:', len(pose_files))

if pose_files:
    sample = pose_files[0]
    schema = infer_schema_from_npy(sample)
    assert schema in ('coco17','openpose25'), f"Unknown schema for {sample}"
    mapping = build_mapping(schema)
    out_csv = f"{LANDMARKS_DIR}/mmfit_pose2d_landmarks.csv"
    os.makedirs(LANDMARKS_DIR, exist_ok=True)
    with open(out_csv,'w',newline='',encoding='utf-8') as f:
        w = csv.writer(f)
        write_header(w)
        for p in pose_files:
            arr = np.load(p)
            if arr.ndim == 2:
                T,twoK = arr.shape
                arr = arr.reshape(T, twoK//2, 2)
            T,K,_ = arr.shape
            video_id = os.path.basename(os.path.dirname(p))  # wXX
            ts = 0.0
            for t in range(0,T,2):  # stride=2
                row=[video_id,t,ts,1,1]
                lm=[None]*(BLAZEPOSE_SIZE*4)
                for src_idx,tgt_idx in mapping.items():
                    if src_idx < K:
                        x,y = float(arr[t,src_idx,0]), float(arr[t,src_idx,1])
                        off=tgt_idx*4
                        lm[off+0]=x; lm[off+1]=y; lm[off+2]=0.0; lm[off+3]=1.0
                row+=lm
                w.writerow(row)
                ts+=33.33
    print('Saved:', out_csv)
else:
    print('No pose_2d.npy files found; skip Option B.')

## 7. Feature Engineering
Reads a landmarks CSV (from Option A or B) and computes per-frame features.

In [ ]:
import pandas as pd, numpy as np, math, os

# choose the landmarks CSV to use
lm_csv_a = f"{LANDMARKS_DIR}/mmfit_videos_landmarks.csv"
lm_csv_b = f"{LANDMARKS_DIR}/mmfit_pose2d_landmarks.csv"
LANDMARK_CSV = lm_csv_b if os.path.exists(lm_csv_b) else lm_csv_a
print('Using landmarks:', LANDMARK_CSV)

# Helpers
IDX = {'nose':0,'l_sh':11,'r_sh':12,'l_el':13,'r_el':14,'l_wr':15,'r_wr':16,'l_hip':23,'r_hip':24,'l_knee':25,'r_knee':26,'l_ank':27,'r_ank':28}

def angle_between(a,b,c):
    ba = a-b; bc = c-b
    if np.any(np.isnan(ba)) or np.any(np.isnan(bc)):
        return np.nan
    nba, nbc = np.linalg.norm(ba), np.linalg.norm(bc)
    if nba==0 or nbc==0:
        return np.nan
    cosang = float(np.dot(ba,bc)/(nba*nbc))
    cosang = max(-1.0,min(1.0,cosang))
    return math.degrees(math.acos(cosang))


df = pd.read_csv(LANDMARK_CSV)
if 'lm0_x' in df.columns:
    df = df.dropna(subset=['lm0_x']).reset_index(drop=True)

rows=[]
for _,row in df.iterrows():
    w,h = float(row['width']), float(row['height'])
    pts = np.full((33,3), np.nan, dtype=float)
    for i in range(33):
        x = row.get(f'lm{i}_x', np.nan); y = row.get(f'lm{i}_y', np.nan); z = row.get(f'lm{i}_z', np.nan)
        if not (pd.isna(x) or pd.isna(y)):
            # If pose2d had width/height=1, keep as-is; otherwise scale by frame dims
            if w>1 and h>1:
                x *= w; y *= h
            pts[i] = (x,y, z if not pd.isna(z) else 0.0)
    # angles
    l_elbow = angle_between(pts[IDX['l_sh']], pts[IDX['l_el']], pts[IDX['l_wr']])
    r_elbow = angle_between(pts[IDX['r_sh']], pts[IDX['r_el']], pts[IDX['r_wr']])
    l_knee = angle_between(pts[IDX['l_hip']], pts[IDX['l_knee']], pts[IDX['l_ank']])
    r_knee = angle_between(pts[IDX['r_hip']], pts[IDX['r_knee']], pts[IDX['r_ank']])
    l_hip = angle_between(pts[IDX['l_sh']], pts[IDX['l_hip']], pts[IDX['l_knee']])
    r_hip = angle_between(pts[IDX['r_sh']], pts[IDX['r_hip']], pts[IDX['r_knee']])

    mid_hip = (pts[IDX['l_hip']]+pts[IDX['r_hip']])/2.0
    nose = pts[IDX['nose']]
    body_tilt = math.degrees(math.atan2(nose[1]-mid_hip[1], nose[0]-mid_hip[0])) if not (np.any(np.isnan(nose)) or np.any(np.isnan(mid_hip))) else np.nan

    def dist(a,b):
        if np.any(np.isnan(a[:2])) or np.any(np.isnan(b[:2])):
            return np.nan
        return float(np.linalg.norm(a[:2]-b[:2]))

    shoulder_width = dist(pts[IDX['l_sh']], pts[IDX['r_sh']])
    hip_width = dist(pts[IDX['l_hip']], pts[IDX['r_hip']])
    torso_len = dist((pts[IDX['l_sh']]+pts[IDX['r_sh']])/2.0, mid_hip)
    arm_len = dist(pts[IDX['l_sh']], pts[IDX['l_wr']])

    ratio_sh_hip = shoulder_width/(hip_width+1e-6) if not (pd.isna(shoulder_width) or pd.isna(hip_width)) else np.nan
    ratio_arm_torso = arm_len/(torso_len+1e-6) if not (pd.isna(arm_len) or pd.isna(torso_len)) else np.nan

    rows.append({
        'video': row['video'], 'frame': int(row['frame']), 'timestamp': float(row['timestamp']),
        'l_elbow': l_elbow, 'r_elbow': r_elbow,
        'l_knee': l_knee, 'r_knee': r_knee,
        'l_hip': l_hip, 'r_hip': r_hip,
        'body_tilt': body_tilt,
        'ratio_sh_hip': ratio_sh_hip,
        'ratio_arm_torso': ratio_arm_torso,
        'shoulder_width': shoulder_width,
        'hip_width': hip_width,
    })

feat_df = pd.DataFrame(rows)
FEAT_PKL = f"{FEATURES_DIR}/mmfit_features.pkl"
feat_df.to_pickle(FEAT_PKL)
print('Saved features:', FEAT_PKL, feat_df.shape)
feat_df.head()

## 8. Create Windows and Labels
Ensure you have a label CSV at `data/video_labels.csv` with columns: video,exercise,form (e.g., `w00,squat,correct`).

In [ ]:
import pandas as pd, numpy as np, os

labels_csv = f"{DATA_DIR}/video_labels.csv"
if not os.path.exists(labels_csv):
    with open(labels_csv,'w') as f:
        f.write('video,exercise,form\n')
        f.write('w00,squat,correct\n')
        f.write('w01,biceps,incorrect\n')
    print('Created a sample labels CSV at', labels_csv)

feat_df = pd.read_pickle(f"{FEATURES_DIR}/mmfit_features.pkl")
labels_df = pd.read_csv(labels_csv)

FPS = 30
WINDOW_SEC = 1.5
WINDOW_SIZE = int(WINDOW_SEC * FPS / 2)  # sample_rate=2 if from videos; same stride used in pose2d conversion
STEP = max(1, WINDOW_SIZE//2)

rows=[]
num_cols=['l_elbow','r_elbow','l_knee','r_knee','l_hip','r_hip','body_tilt','ratio_sh_hip','ratio_arm_torso']
for video,vdf in feat_df.groupby('video'):
    vdf=vdf.sort_values('frame').reset_index(drop=True)
    for start in range(0, len(vdf)-WINDOW_SIZE+1, STEP):
        win=vdf.iloc[start:start+WINDOW_SIZE]
        agg={'video': video, 'start_frame': int(win.iloc[0]['frame'])}
        for c in num_cols:
            agg[f'{c}_mean']=float(win[c].mean())
            agg[f'{c}_std']=float(win[c].std())
            agg[f'{c}_min']=float(win[c].min())
            agg[f'{c}_max']=float(win[c].max())
        rows.append(agg)

win_df = pd.DataFrame(rows)
win_df = win_df.merge(labels_df, on='video', how='left')
WIN_PKL = f"{FEATURES_DIR}/mmfit_windows.pkl"
win_df.to_pickle(WIN_PKL)
print('Saved windows:', WIN_PKL, win_df.shape)
win_df.head()

## 9. Train Models and Visualize
Trains RandomForest baseline vs enhanced and saves confusion matrices to `results/`.

In [ ]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns, matplotlib.pyplot as plt, os

WIN_PKL = f"{FEATURES_DIR}/mmfit_windows.pkl"
df = pd.read_pickle(WIN_PKL)

df = df.dropna(subset=['exercise']).reset_index(drop=True)
le = LabelEncoder()
df['y'] = le.fit_transform(df['exercise'])

videos = df['video'].unique()
train_vids, test_vids = train_test_split(videos, test_size=0.2, random_state=42)
train_df = df[df['video'].isin(train_vids)]
test_df = df[df['video'].isin(test_vids)]

baseline_cols = ['l_elbow_mean','r_elbow_mean','l_knee_mean','r_knee_mean','l_hip_mean','r_hip_mean']
enhanced_cols = baseline_cols + ['body_tilt_mean','ratio_sh_hip_mean','ratio_arm_torso_mean'] + [c for c in df.columns if c.endswith('_std')]

# Train baseline
Xb_tr = train_df[baseline_cols].fillna(0); y_tr = train_df['y']
Xb_te = test_df[baseline_cols].fillna(0); y_te = test_df['y']
rf_b = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
rf_b.fit(Xb_tr, y_tr)
yp_b = rf_b.predict(Xb_te)
print('Baseline report\n', classification_report(y_te, yp_b, target_names=le.classes_))

# Train enhanced
Xe_tr = train_df[enhanced_cols].fillna(0)
Xe_te = test_df[enhanced_cols].fillna(0)
rf_e = RandomForestClassifier(n_estimators=300, max_depth=20, random_state=42, n_jobs=-1)
rf_e.fit(Xe_tr, y_tr)
yp_e = rf_e.predict(Xe_te)
print('Enhanced report\n', classification_report(y_te, yp_e, target_names=le.classes_))

# Save confusion matrices
os.makedirs(RESULTS_DIR, exist_ok=True)
cm_b = confusion_matrix(y_te, yp_b)
cm_e = confusion_matrix(y_te, yp_e)
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); sns.heatmap(cm_b, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_); plt.title('Baseline')
plt.subplot(1,2,2); sns.heatmap(cm_e, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_); plt.title('Enhanced')
plt.tight_layout()
out_png = f"{RESULTS_DIR}/confusion_comparison.png"
plt.savefig(out_png, dpi=200)
print('Saved figure:', out_png)
plt.show()

## 10. Rep Counting Demo
Counts reps using simple thresholds over angle sequences.

In [ ]:
import pandas as pd

feat_df = pd.read_pickle(f"{FEATURES_DIR}/mmfit_features.pkl")

# Example: count biceps reps via left elbow angle thresholds
v = feat_df['video'].iloc[0] if len(feat_df)>0 else None
if v is not None:
    vdf = feat_df[feat_df['video']==v].sort_values('frame')
    ang = vdf['l_elbow'].fillna(method='ffill').fillna(method='bfill').values
    state='up'; reps=0
    for a in ang:
        if state=='up' and a<50:
            state='down'
        elif state=='down' and a>160:
            state='up'; reps+=1
    print(f'Estimated reps for {v}:', reps)
else:
    print('No features found; ensure earlier steps ran.')

## 11. Persist Results and Artifacts
List outputs and optionally zip them for download.

In [ ]:
import os

!ls -R {RESULTS_DIR} {MODELS_DIR} {FEATURES_DIR} {LANDMARKS_DIR}
# Optional: create a bundle
#!zip -r {BASE_DIR}/results_bundle.zip {RESULTS_DIR} {MODELS_DIR}

## 12. Session Info and Reproducibility
Record the environment and freeze packages for reproducibility.

In [ ]:
import sys, platform, os
print(sys.version)
print(platform.platform())
!pip freeze | tee {BASE_DIR}/requirements_colab.txt
print('Saved requirements to', f'{BASE_DIR}/requirements_colab.txt')